# 1. 环境准备

## 1.1 python 环境准备

请运行以下代码完成 python 环境的安装：

In [ ]:
! pip install gradio==6.1.0 openai==2.11.0

## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

## 1.3 大模型调用函数准备

若想了解更多关于函数部分的内容，请查阅 **2.1 函数** 部分内容。

In [ ]:
import os
from openai import OpenAI

def llm_response(content):
  client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY"), 
  base_url="https://aistudio.baidu.com/llm/lmapi/v3", 
  )

  chat_completion = client.chat.completions.create(
  messages=[
    {'role': 'system', 'content': '你是 AI Studio 实训AI开发平台的开发者助理，你精通开发相关的知识，负责给开发者提供搜索帮助建议。'},
    {'role': 'user', 'content': content}
  ],
  model="ernie-3.5-8k",
  )

  return chat_completion.choices[0].message.content

## 1.3 辅助工具简介

除了前面提到的组件以及页面构建以外，gradio 里还为我们准备了许多辅助的小工具帮助我们更好的完成任务，这里包括：
- 弹窗系统（Modal）：Gradio 内置了完整的弹窗系统，用于在应用运行过程中向用户显示即时反馈。包括 —— Error（错误弹窗）、Info（信息弹窗）、Waring（警告弹窗）。
- 进度条工具（Progress）：Progress 是 Gradio 内置的“后端进度条”系统，支持将多步骤任务通过进度条形式在页面中展示。
- 主题（Theming）：Gradio 内置多个预设主题（Theme），允许你快速切换应用的整体视觉风格，包括颜色、边角、阴影、字体等。

# 2. 弹窗系统

## 2.1 简介

关于弹窗系统，主要分为三个类型：
- Error（错误弹窗）：一般使用红色样式，表示严重错误 / 阻断流程。常用于输入无效、模型异常、文件格式错误等。
- Info（信息弹窗）：一般使用灰色样式，常用于普通提示、成功通知、系统信息。
- Warning（警告弹窗）：一般使用黄色样式，提醒用户当前操作可能有风险。多用于删除确认、危险操作提示。

## 2.2 Error（错误弹窗）

我们可以在任意的位置使用以传递自定义的错误信息：

```python
raise gr.Error(message="自定义消息")
```

当这行代码被执行时，该自定义消息会以一个弹窗（modal）的形式显示在界面上。

我们可以看一个实际的简单例子：

In [ ]:
import gradio as gr

def divide(numerator, denominator):
  if denominator == 0:
    raise gr.Error("Cannot divide by zero!")

demo = gr.Interface(divide, ["number", "number"], "number")
demo.launch()

当此时 denominator 的值为 0，我们点击 Submit 的话就会显示报错的信息。

并且由于默认的 print_exception 为 True，在下面可以看到报错信息打印出来：

```
File "C:\Users\76391\AppData\Local\Temp\ipykernel_35240\2585216174.py", line 4, in divide  
raise gr.Error("Cannot divide by zero!") 
gradio.exceptions.Error: 'Cannot divide by zero!'
```


## 2.3 Info（信息弹窗）

我们同样可以在各个位置传入 Info 信息，包括在函数中，也包括中页面创建时。

当然不一样的是不需要通过 raise 的方式来展示，直接写入即可。具体创建方式如下：

```python
gradio.Info(message="Helpful info message", duration=5)
```

我们可以通过一个简单的例子演示一下：

In [ ]:
import gradio as gr

def show_info():
  gr.Info("提示：这是一个 Info 弹窗！")
  return "函数已执行完毕"

demo = gr.Interface(fn=show_info, inputs=None, outputs="text",)

demo.launch() 

当我们点击 Generate 按钮后，此时 Info 的信息就会显示出来。所以可以看到，只要直接将 gr.Info() 写入到运行的函数中，就会将这部分信息显示，并且还不会影响正常的输出组件。

## 2.4 Warning（警告弹窗）

我们同样可以在各个位置传入 Waring 信息，包括在函数中，也包括中页面创建时。

和 Info 一样，我们也不需要通过 raise 来传入。具体创建方式如下：

```python
gr.Warning(message="message here")
```

所以我只需要将前面 Info 的代码改为 Waring 同样可以运行：

In [ ]:
import gradio as gr

def show_info():
  gr.Warning("提示：这是一个 Warning 弹窗！")
  return "函数已执行完毕"

demo = gr.Interface(fn=show_info, inputs=None, outputs="text",)

demo.launch() 

当我们点击 Generate 按钮后，此时 Warning 的信息就会显示出来。所以同样可以看到，只要直接将 gr.Warning() 写入到运行的函数中，就会将这部分信息显示，并且还不会影响正常的输出组件。

# 3. 进度条工具（Progress）

## 3.1 简介

通俗来说，gr.Progress 就是 Gradio 内置的进度条控制器，让你在后端代码运行时，实时把进度发到前端显示。

比如运行某个程序时可能要等十几秒才能回复，此时用户并不知道是卡死了还是还在运行。

用了 Progress 之后：
- 可以在一开始显示 “正在初始化…”
- 循环中显示 0% → 10% → 50% → 100%
- 用户能看见任务在动，体验会好很多

所以进度条工具虽然不会影响正常的程序运行，但是会增强用户体验，这在心理学层面上也是受到验证的（能看到进度条就不会那么焦虑，即使这个进度条是假的）

## 3.2 创建方式

假如想要使用进度条工具，我们需要在函数创建时，在最后面添加一个参数 progress=gr.Progress()。

```python
def my_function(x, progress=gr.Progress(track_tqdm=False)):
```

这样的话，在函数内部，你就可以把 progress 当一个进度调整器使用。

其主要使用方式有两种：
- progress(...)：手动更新进度条
- progress.tqdm(iterable)：包一层循环 iterable，自动更新进度。假如使用原生 tqdm 时需要设置 track_tqdm=True

下面我们来详细说一下两种方法的具体使用方式。


### 3.2.1 手动更新进度条

我们可以在特定的步骤中手动添加进度信息：

In [ ]:
import gradio as gr
import time

def long_task(name, progress=gr.Progress()):
  progress(0.0, desc="开始处理…")
  time.sleep(1)
  progress(0.3, desc="正在加载数据…")
  time.sleep(1)
  progress(0.6, desc="正在处理数据…")
  time.sleep(1)
  progress(1.0, desc="处理完成！")
  return f"你好，{name}！任务已经完成。"

demo = gr.Interface(fn=long_task, inputs="text", outputs="text")
demo.launch()

### 3.2.2 自动更新进度条

假如我们有明确循环次数的任务（如 range(100) ），这个时候可以将其放入到 progress.tqdm 中，这样就可以根据自动跟新进度条：

In [ ]:
import gradio as gr
import time

def auto_progress_task(n, progress=gr.Progress()):
  # tqdm 自动更新进度条
  for i in progress.tqdm(range(n), desc="处理中…"):
    time.sleep(1)
  return f"循环了 {n} 次，全部完成！"

demo = gr.Interface(fn=auto_progress_task, inputs=gr.Number(label="循环次数"), outputs="text")

demo.launch()

假如我们用的不是 progress.tqdm() 而是原生的 tqdm 方法，此时我们需要在函数传入参数的 progress 中添加 track_tqdm=True：

In [ ]:
import gradio as gr
import time
from tqdm import tqdm

def auto_progress_task(n, progress=gr.Progress(track_tqdm=True)):
  for i in tqdm(range(n), desc="处理中…"): # 原生 tqdm
    time.sleep(1)
  return f"循环了 {n} 次，全部完成！"

demo = gr.Interface(
  fn=auto_progress_task, inputs=gr.Number(label="循环次数"), outputs="text")

demo.launch()

假如设置的是 track_tqdm=False ，虽然不会影响程序正常运行，但是进度信息只会在终端打印，而不会在 Gradio 中显示出来。

除了前面看到的 iterable 和 desc 参数外，在函数内的 progress 还有其他两个参数，包括：
- total：估计总共要运行多少步。假如实际只有 5 步，但是我们这里设置了 10 步的话，进度条会优先显示有 10 步，但是到第五步的时候就会停下来。不过大部分情况下不需要进行设置。
- unit：默认值是"steps"，这个是进度条中每个单位的名称。我们可以调整这个名称为别的内容。

# 4. 主题（Theming）

## 4.1 简介

Gradio 主题是一套集中管理的 UI 配置（颜色、圆角、间距、字体、控件样式等），可以给整个应用套皮。

我们在 Blocks 或 Interface 上加一个 theme= 参数，整个 App 的风格就换掉了。

比如在 gr.Blocks() 中：
```python
with gr.Blocks(theme=gr.themes.Soft()) as demo:
```

又或者在 gr.Interface() 中：
```python
demo = gr.Interface(..., theme=gr.themes.Glass())
```


那像 gr.themes.Soft() 这种现成的主题样式还有很多，包括：
- gr.themes.Base()：蓝色
- gr.themes.Default()：橙色
- gr.themes.Glass()：玻璃色
- gr.themes.Monochrome()：黑白传统
- gr.themes.Soft()：蓝紫色

## 4.2 theme builder

在 Gradio 中还提供给我们低代码可视化改主题的工具 gr.themes.builder()

我们可以直接在 .py 文件中运行以下代码即可打开：

In [ ]:
# 建议先降级 gradio 版本到 5.49.1，不然看不到效果

import gradio as gr
gr.themes.builder()

启动后会打开一个专门的“主题编辑器”App。可以在线调颜色、圆角、间距、字体等，并且可以实时预览效果。

最后可以直接导出生成这个主题的代码（直接复制到你的项目就行）或上传到 hub 中(当然我们也可以使用别人上传 Hub 的主题 theme=gr.Theme.from_hub("...") )：

这样我们不仅可以直观看到 Gradio 内的样式变化，还可以将这个变化同步到我们自己的页面当中！非常适合想做一个“本课程专属主题”，但又不想自己研究具体代码的时候。